In [1]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
plt.rcParams['axes.unicode_minus']=False
plt.rcParams['font.sans-serif']=['SimHei']
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import openpyxl


#加载构建CNN网络的包
from tensorflow.keras.layers import Dense,RNN,LSTM,Activation,Dropout,Conv2D,MaxPooling2D,Flatten,BatchNormalization
from tensorflow.keras.layers import Embedding, Flatten, Dense, LSTM
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import RMSprop,Adam
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
from tensorflow.keras.utils import plot_model

from sklearn.decomposition import PCA
import collections
from itertools import cycle
from scipy import interp

from sklearn.metrics import classification_report, accuracy_score,auc,confusion_matrix,precision_score,recall_score,roc_auc_score,\
roc_curve,f1_score
from sklearn import metrics
import pickle
import json
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
files = os.listdir('拉曼数据')
# data0 = pd.read_table('拉曼数据/'+'4-1-1.txt',header=None)
# dat = np.zeros((len(files),data0.shape[0],data0.shape[1]))
# for i in range(len(files)):
#     data0 = pd.read_table('拉曼数据/'+files[i],header=None)
#     dat[i,:,:] = data0.values
# dat = dat.reshape((dat.shape[0],dat.shape[1],dat.shape[2],1))
# dat.shape

In [3]:
#定义绘制CNN模型训练过程准确率和损失率的函数
def plot1(history,file=''):
    acc = history.history['mse']#训练集的准确率
    val_acc = history.history['val_mse']#验证集的准确率
    loss = history.history['loss']#训练集的过程损失
    val_loss = history.history['val_loss']#验证集的过程损失
    
    epochs = range(1, len(acc) + 1)#迭代次数
    
    #下面分别画出图并保存
    plt.plot(epochs, acc, 'r', label='Training mse')
    plt.plot(epochs, val_acc, 'b', label='Validation mse')
    plt.title('Training and Validation mse')
    plt.xlabel('epochs')
    plt.ylabel('mse')
    # plt.savefig(file+'Training and Validation mse.png',bbox='tight')
    plt.legend()
    plt.show()
    
    plt.figure()
    plt.plot(epochs, loss, 'r', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and Validation loss')
    plt.xlabel('epochs')
    plt.ylabel('loss')
    plt.legend()
    # plt.savefig(file+'Training and Validation loss.png',bbox='tight')
    plt.show()

In [4]:
# data.head(10)
data=pd.read_excel("newlabel.xlsx",index_col='序号',header=0)
data.head()

DSS       AWS      MDXS      DFSB        Sc
序号                                                     
4-1-1  0.047086  0.031652  0.105118  2.054229  3.503772
4-1-2  0.052025  0.009681  0.112967  2.177835  3.919927
4-1-3  0.056221  0.029940  0.116185  2.251329  3.998878
4-1-4  0.059036  0.035721  0.118939  2.298153  4.159827
4-1-5  0.062070  0.045199  0.121587  2.320460  4.189648

In [5]:
y = pd.DataFrame()
y['序号'] = [i[:-4] for i in files]
y = pd.merge(y,data,on='序号')
y.head()

序号       DSS       AWS      MDXS      DFSB        Sc
0   10-1-1  0.039384  0.031712  0.100845  1.736759  2.891318
1  10-1-10  0.067615  0.051542  0.147246  2.393790  3.942209
2  10-1-11  0.068828  0.052690  0.147067  2.393317  3.981053
3  10-1-12  0.072014  0.055386  0.154242  2.471002  4.043494
4  10-1-13  0.075722  0.058593  0.156851  2.486423  4.160139

In [6]:
res = pd.DataFrame(np.zeros((y.shape[1]-1,4)),index=y.columns[1:],columns=['tain r2','test r2','tain RMSE','test RMSE'])

In [7]:
res.index

Index(['DSS', 'AWS', 'MDXS', 'DFSB', 'Sc'], dtype='object')

In [8]:
def plotrealpred(yc,ycp,yv,yvp):
    plt.figure(figsize=(5, 5), dpi=100)
    
    plt.scatter(yc, ycp, s=5, c='r', marker='o',alpha=0.65)
    plt.scatter(yv, yvp, s=5, c='b', marker='o',alpha=0.65)
    plt.legend(["train",'test'])
    plt.plot([-10,10],[-10,10])
    plt.xlabel('Measured mg/ml')
    plt.ylabel('Predicted mg/ml')
    plt.xlim(min(yc)-0.03, max(yc)+0.03)
    plt.ylim(min(yc)-0.03, max(yc)+0.03)    
    # plt.tight_layout(pad=0.3)
    # plt.grid(True)
    return plt

import random

def plottime(yc,ycp,yv,yvp):
    pass
    # return plt

In [9]:
dff=pd.read_excel("x_y_new.xlsx",index_col='name',header=0)
dff.shape

(325, 1909)

In [12]:
for i in dff.index:
    df=dff.drop([i],inplace = False)
    print(df.shape)

(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324, 1909)
(324

In [ ]:
for i in dff.index:
    df=dff.drop([i],inplace = False)
    print(df.shape)

    x=df.iloc[:,:-5]
    DSS=df['DSS/mg/ml']
    AWS=df['AWS/mg/ml']
    MDXS=df['MDXS/mg/ml']
    DFSB=df['DFSB/mg/ml']
    Sc=df['Sc/%']

    import CARS
    lis = CARS.CARS_Cloud(np.array(x), np.array(DSS))
    x=np.array(x)
    x_=x[:,lis]
    x_train, x_test, y_train, y_test = train_test_split(x_, np.array(DSS), test_size=0.2, random_state=45)
    x_train_=np.expand_dims(x_train,2).repeat(2,axis=2)
    x_train_=np.expand_dims(x_train_,3)
    x_train=x_train_
    x_test_=np.expand_dims(x_test,2).repeat(2,axis=2)
    x_test_=np.expand_dims(x_test_,3)
    x_test=x_test_
    y_train_=np.expand_dims(y_train,1)
    y_train=y_train_
    y_test_=np.expand_dims(y_test,1)
    y_test=y_test_
    c = 'DSS'
    
    model_cnn = Sequential()
    model_cnn.add(Conv2D(32, (3, 1), activation='relu',input_shape=(x_train.shape[1],x_train.shape[2],1)))
    model_cnn.add(BatchNormalization(input_dim=(x_train.shape[1],x_train.shape[2],1)))
    model_cnn.add(MaxPooling2D((2,1)))
    #     model_cnn.add(Dropout(0.4))
    model_cnn.add(Conv2D(16, (3, 1), activation='relu'))
    model_cnn.add(Conv2D(16, (3, 1), activation='relu'))
    model_cnn.add(Conv2D(16, (3, 1), activation='relu'))
    model_cnn.add(Conv2D(16, (3, 1), activation='relu'))
    model_cnn.add(Conv2D(16, (3, 1), activation='relu'))
    model_cnn.add(Conv2D(32, (3, 1), activation='relu'))
    model_cnn.add(Conv2D(64, (3, 1), activation='relu'))
    model_cnn.add(Flatten())
    model_cnn.add(Dense(64, activation='relu'))
    model_cnn.add(Dense(1, activation='linear'))
    model_cnn.compile(loss='mean_squared_error',optimizer=Adam(lr=1e-4),metrics=['mse'])
    callbacks = [EarlyStopping(monitor='val_loss', patience=40, verbose=1), # 当patience次迭代损失未改善，Keras停止训练
                    ]
    history_cnn = model_cnn.fit(x_train,y_train,batch_size=50,epochs=200,validation_data=(x_test, y_test),verbose=1,callbacks=callbacks)
    model_cnn.save(c+"___"+"cnn.h5")
    print(metrics.r2_score(y_test,model_cnn.predict(x_test)))
    res.loc[c,'test r2'] = metrics.r2_score(y_test,model_cnn.predict(x_test))
    res.loc[c,'tain r2'] = metrics.r2_score(y_train,model_cnn.predict(x_train))
    res.loc[c,'test RMSE'] = np.sqrt(metrics.mean_squared_error(model_cnn.predict(x_test),y_test))
    res.loc[c,'tain RMSE'] = np.sqrt(metrics.mean_squared_error(model_cnn.predict(x_train),y_train))
        
    plot1(history_cnn,file=c+'______')

    y_predictions = model_cnn.predict(x_train)
    y_predictions_p = model_cnn.predict(x_test)
    plotrealpred(y_train,y_predictions,y_test,y_predictions_p)
    plottime(y_train,y_predictions,y_test,y_predictions_p)

    test_size = 0.2
    random_state = 1

    c = 'AWS'

    import CARS
    lis = CARS.CARS_Cloud(np.array(x), np.array(AWS))
    x=np.array(x)
    x_=x[:,lis]
    x_train, x_test, y_train, y_test = train_test_split(x_, np.array(AWS), test_size=0.2, random_state=45)
    x_train_=np.expand_dims(x_train,2).repeat(2,axis=2)
    x_train_=np.expand_dims(x_train_,3)
    x_train=x_train_
    x_test_=np.expand_dims(x_test,2).repeat(2,axis=2)
    x_test_=np.expand_dims(x_test_,3)
    x_test=x_test_
    y_train_=np.expand_dims(y_train,1)
    y_train=y_train_
    y_test_=np.expand_dims(y_test,1)
    y_test=y_test_
    model_cnn = Sequential()
    model_cnn.add(Conv2D(64, (3, 1), activation='relu',input_shape=(x_train.shape[1],x_train.shape[2],1)))
    model_cnn.add(BatchNormalization(input_dim=(x_train.shape[1],x_train.shape[2],1)))
    # model_cnn.add(MaxPooling2D((2,1)))
    #     model_cnn.add(Dropout(0.4))
    model_cnn.add(Conv2D(16, (3, 1), activation='relu'))
    model_cnn.add(Conv2D(16, (3, 1), activation='relu'))
    model_cnn.add(Conv2D(16, (3, 1), activation='relu'))
    model_cnn.add(Conv2D(16, (3, 1), activation='relu'))
    model_cnn.add(Conv2D(16, (3, 1), activation='relu'))
    model_cnn.add(Conv2D(32, (3, 1), activation='relu'))
    model_cnn.add(Conv2D(64, (3, 1), activation='relu'))
    model_cnn.add(Conv2D(64, (3, 1), activation='relu'))
    model_cnn.add(Flatten())
    model_cnn.add(Dense(64, activation='relu'))
    model_cnn.add(Dense(16, activation='relu'))
    model_cnn.add(Dense(1, activation='linear'))
    model_cnn.compile(loss='mean_squared_error',optimizer=Adam(lr=1e-4),metrics=['mse'])
    callbacks = [EarlyStopping(monitor='val_loss', patience=40, verbose=1), # 当patience次迭代损失未改善，Keras停止训练
                    ]
    history_cnn = model_cnn.fit(x_train,y_train,batch_size=10,epochs=200,validation_data=(x_test, y_test),verbose=1,callbacks=callbacks)
    model_cnn.save(c+"___"+"cnn.h5")
    print(metrics.r2_score(y_test,model_cnn.predict(x_test)))

    c='AWS'
    res.loc[c,'test r2'] = metrics.r2_score(y_test,model_cnn.predict(x_test))
    res.loc[c,'tain r2'] = metrics.r2_score(y_train,model_cnn.predict(x_train))
    res.loc[c,'test RMSE'] = np.sqrt(metrics.mean_squared_error(model_cnn.predict(x_test),y_test))
    res.loc[c,'tain RMSE'] = np.sqrt(metrics.mean_squared_error(model_cnn.predict(x_train),y_train))
        
    plot1(history_cnn,file=c+'______')

    y_predictions = model_cnn.predict(x_train)
    y_predictions_p = model_cnn.predict(x_test)
    plotrealpred(y_train,y_predictions,y_test,y_predictions_p)
    plottime(y_train,y_predictions,y_test,y_predictions_p)


    test_size = 0.2
    random_state = 1

    c = 'MDXS'

    import CARS
    lis = CARS.CARS_Cloud(np.array(x), np.array(MDXS))
    x=np.array(x)
    x_=x[:,lis]
    x_train, x_test, y_train, y_test = train_test_split(x_, np.array(MDXS), test_size=0.2, random_state=45)
    x_train_=np.expand_dims(x_train,2).repeat(2,axis=2)
    x_train_=np.expand_dims(x_train_,3)
    x_train=x_train_
    x_test_=np.expand_dims(x_test,2).repeat(2,axis=2)
    x_test_=np.expand_dims(x_test_,3)
    x_test=x_test_
    y_train_=np.expand_dims(y_train,1)
    y_train=y_train_
    y_test_=np.expand_dims(y_test,1)
    y_test=y_test_
    model_cnn = Sequential()
    model_cnn.add(Conv2D(16, (3, 1), activation='relu',input_shape=(x_train.shape[1],x_train.shape[2],1)))
    model_cnn.add(BatchNormalization(input_dim=(x_train.shape[1],x_train.shape[2],1)))
    model_cnn.add(MaxPooling2D((2,1)))
    #     model_cnn.add(Dropout(0.4))
    model_cnn.add(Conv2D(16, (3, 1), activation='relu'))
    model_cnn.add(Conv2D(16, (3, 1), activation='relu'))
    model_cnn.add(Conv2D(16, (3, 1), activation='relu'))
    model_cnn.add(Conv2D(16, (3, 1), activation='relu'))
    model_cnn.add(Conv2D(32, (3, 1), activation='relu'))
    model_cnn.add(Conv2D(64, (3, 1), activation='relu'))
    model_cnn.add(Flatten())
    model_cnn.add(Dense(64, activation='relu'))
    model_cnn.add(Dense(16, activation='relu'))
    model_cnn.add(Dense(1, activation='linear'))
    model_cnn.compile(loss='mean_squared_error',optimizer=Adam(lr=1e-4),metrics=['mse'])
    callbacks = [EarlyStopping(monitor='val_loss', patience=40, verbose=1), # 当patience次迭代损失未改善，Keras停止训练
                    ]
    history_cnn = model_cnn.fit(x_train,y_train,batch_size=50,epochs=200,validation_data=(x_test, y_test),verbose=1,callbacks=callbacks)
    model_cnn.save(c+"___"+"cnn.h5")
    print(metrics.r2_score(y_test,model_cnn.predict(x_test)))
    res.loc[c,'test r2'] = metrics.r2_score(y_test,model_cnn.predict(x_test))
    res.loc[c,'tain r2'] = metrics.r2_score(y_train,model_cnn.predict(x_train))
    res.loc[c,'test RMSE'] = np.sqrt(metrics.mean_squared_error(model_cnn.predict(x_test),y_test))
    res.loc[c,'tain RMSE'] = np.sqrt(metrics.mean_squared_error(model_cnn.predict(x_train),y_train))
        
    plot1(history_cnn,file=c+'______')

    y_predictions = model_cnn.predict(x_train)
    y_predictions_p = model_cnn.predict(x_test)
    plotrealpred(y_train,y_predictions,y_test,y_predictions_p)
    plottime(y_train,y_predictions,y_test,y_predictions_p)


    test_size = 0.2
    random_state = 1

    c = 'DFSB'

    import CARS
    lis = CARS.CARS_Cloud(np.array(x), np.array(DFSB))
    x=np.array(x)
    x_=x[:,lis]
    x_train, x_test, y_train, y_test = train_test_split(x_, np.array(DFSB), test_size=0.2, random_state=45)
    x_train_=np.expand_dims(x_train,2).repeat(2,axis=2)
    x_train_=np.expand_dims(x_train_,3)
    x_train=x_train_
    x_test_=np.expand_dims(x_test,2).repeat(2,axis=2)
    x_test_=np.expand_dims(x_test_,3)
    x_test=x_test_
    y_train_=np.expand_dims(y_train,1)
    y_train=y_train_
    y_test_=np.expand_dims(y_test,1)
    y_test=y_test_
    x_train.shape,x_test.shape,y_train.shape,y_test.shape,
        
    model_cnn = Sequential()
    model_cnn.add(Conv2D(16, (3, 1), activation='relu',input_shape=(x_train.shape[1],x_train.shape[2],1)))
    model_cnn.add(BatchNormalization(input_dim=(x_train.shape[1],x_train.shape[2],1)))
    model_cnn.add(MaxPooling2D((2,1)))
    #     model_cnn.add(Dropout(0.4))
    model_cnn.add(Conv2D(16, (3, 1), activation='relu'))
    model_cnn.add(Conv2D(32, (8, 1), activation='relu'))
    model_cnn.add(Conv2D(64, (8, 1), activation='relu'))
    model_cnn.add(Flatten())
    model_cnn.add(Dense(64, activation='relu'))
    model_cnn.add(Dense(1, activation='linear'))
    model_cnn.compile(loss='mean_squared_error',optimizer=Adam(lr=1e-4),metrics=['mse'])
    callbacks = [EarlyStopping(monitor='val_loss', patience=40, verbose=1), # 当patience次迭代损失未改善，Keras停止训练
                    ]
    history_cnn = model_cnn.fit(x_train,y_train,batch_size=50,epochs=500,validation_data=(x_test, y_test),verbose=1,callbacks=callbacks)
    model_cnn.save(c+"___"+"cnn.h5")
    print(metrics.r2_score(y_test,model_cnn.predict(x_test)))
    res.loc[c,'test r2'] = metrics.r2_score(y_test,model_cnn.predict(x_test))
    res.loc[c,'tain r2'] = metrics.r2_score(y_train,model_cnn.predict(x_train))
    res.loc[c,'test RMSE'] = np.sqrt(metrics.mean_squared_error(model_cnn.predict(x_test),y_test))
    res.loc[c,'tain RMSE'] = np.sqrt(metrics.mean_squared_error(model_cnn.predict(x_train),y_train))
        
    plot1(history_cnn,file=c+'______')

    y_predictions = model_cnn.predict(x_train)
    y_predictions_p = model_cnn.predict(x_test)
    plotrealpred(y_train,y_predictions,y_test,y_predictions_p)
    plottime(y_train,y_predictions,y_test,y_predictions_p)


    test_size = 0.2
    random_state = 1

    c = 'Sc'

    import CARS
    lis = CARS.CARS_Cloud(np.array(x), np.array(Sc))
    x=np.array(x)
    x_=x[:,lis]
    x_train, x_test, y_train, y_test = train_test_split(x_, np.array(Sc), test_size=0.2, random_state=45)
    x_train_=np.expand_dims(x_train,2).repeat(2,axis=2)
    x_train_=np.expand_dims(x_train_,3)
    x_train=x_train_
    x_test_=np.expand_dims(x_test,2).repeat(2,axis=2)
    x_test_=np.expand_dims(x_test_,3)
    x_test=x_test_
    y_train_=np.expand_dims(y_train,1)
    y_train=y_train_
    y_test_=np.expand_dims(y_test,1)
    y_test=y_test_
    model_cnn = Sequential()
    model_cnn.add(Conv2D(16, (3, 1), activation='relu',input_shape=(x_train.shape[1],x_train.shape[2],1)))
    model_cnn.add(BatchNormalization(input_dim=(x_train.shape[1],x_train.shape[2],1)))
    model_cnn.add(MaxPooling2D((2,1)))
    #     model_cnn.add(Dropout(0.4))
    model_cnn.add(Conv2D(16, (8, 1), activation='relu'))
    model_cnn.add(Conv2D(32, (8, 1), activation='relu'))
    model_cnn.add(Conv2D(64, (8, 1), activation='relu'))
    model_cnn.add(Flatten())
    model_cnn.add(Dense(64, activation='relu'))
    model_cnn.add(Dense(1, activation='linear'))
    model_cnn.compile(loss='mean_squared_error',optimizer=Adam(lr=1e-4),metrics=['mse'])
    callbacks = [EarlyStopping(monitor='val_loss', patience=40, verbose=1), # 当patience次迭代损失未改善，Keras停止训练
                    ]
    history_cnn = model_cnn.fit(x_train,y_train,batch_size=50,epochs=500,validation_data=(x_test, y_test),verbose=1,callbacks=callbacks)
    model_cnn.save(c+"___"+"cnn.h5")
    print(metrics.r2_score(y_test,model_cnn.predict(x_test)))
    res.loc[c,'test r2'] = metrics.r2_score(y_test,model_cnn.predict(x_test))
    res.loc[c,'tain r2'] = metrics.r2_score(y_train,model_cnn.predict(x_train))
    res.loc[c,'test RMSE'] = np.sqrt(metrics.mean_squared_error(model_cnn.predict(x_test),y_test))
    res.loc[c,'tain RMSE'] = np.sqrt(metrics.mean_squared_error(model_cnn.predict(x_train),y_train))
        
    plot1(history_cnn,file=c+'______')

    y_predictions = model_cnn.predict(x_train)
    y_predictions_p = model_cnn.predict(x_test)
    plotrealpred(y_train,y_predictions,y_test,y_predictions_p)
    plottime(y_train,y_predictions,y_test,y_predictions_p)

    res.to_excel(i+'.xlsx')
